<a href="https://colab.research.google.com/github/premkumar25/Movie-Recommendation-KNN/blob/main/Movie_Recommendation_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
%matplotlib inline
warnings.filterwarnings('ignore')

In [3]:
movies_df = pd.read_csv('movies.txt')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
rating_df = pd.read_csv('rating.txt')
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
df = pd.merge(movies_df, rating_df, on='movieId')
df.drop(['timestamp','genres'],axis=1, inplace=True)
df.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),1,4.0
1,1,Toy Story (1995),5,4.0
2,1,Toy Story (1995),7,4.5
3,1,Toy Story (1995),15,2.5
4,1,Toy Story (1995),17,4.5


In [6]:
df.isnull().sum()

movieId    0
title      0
userId     0
rating     0
dtype: int64

### Grouping by the title and the count of review for each movie

In [7]:
movie_rating_count = (df.groupby(by=['title'])['rating'].count().reset_index().rename(columns={'rating':'total_rating_count'}))
movie_rating_count.head()

,title,total_rating_count
0,'Hellboy': The Seeds of Creation (2004),1
1,'Round Midnight (1986),2
2,'Salem's Lot (2004),1
3,'Til There Was You (1997),2
4,'Tis the Season for Love (2015),1


In [8]:
total_movie_rating_count_df = pd.merge(df, movie_rating_count, on='title')
total_movie_rating_count_df.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,205
1,1,Toy Story (1995),5,4.0,205
2,1,Toy Story (1995),7,4.5,205
3,1,Toy Story (1995),15,2.5,205
4,1,Toy Story (1995),17,4.5,205


In [9]:
total_movie_rating_count_df.describe()

,movieId,userId,rating,total_rating_count
count,93228.000000,93228.000000,93228.000000,93228.000000
mean,19318.053771,303.447108,3.521603,57.761788
std,35491.159175,171.031434,1.042340,60.677294
min,1.000000,1.000000,0.500000,1.000000
25%,1193.000000,160.000000,3.000000,13.000000
50%,2947.000000,305.000000,3.500000,38.000000
75%,7701.000000,448.000000,4.000000,83.000000
max,193609.000000,599.000000,5.000000,321.000000


## Set the popularity value

Movies with rating higher than popularity value will only be considered. This helps in reducing the wrong recommendation like if one movie has only one review that wont be considered

In [10]:
popularity_level =  50
popular_movie_rating = total_movie_rating_count_df.query('total_rating_count>=@popularity_level')
popular_movie_rating.head()

,movieId,title,userId,rating,total_rating_count
0,1,Toy Story (1995),1,4.0,205
1,1,Toy Story (1995),5,4.0,205
2,1,Toy Story (1995),7,4.5,205
3,1,Toy Story (1995),15,2.5,205
4,1,Toy Story (1995),17,4.5,205


### Convert to pivot table

In [11]:
movie_pivot_df = popular_movie_rating.pivot_table(values='rating',index='title',columns='userId').fillna(0)
movie_pivot_df.head()

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599
title,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,2.0,3.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.5,0.0,0.0,0.0,0.0,0.0,3.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,3.5,0.0,0.0,4.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,3.5,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Convert the pivot table to np array

In [12]:
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors 

Here we are using cosine similarity to identify the distance between two movies

In [14]:
movie_pivot_df_matrix = csr_matrix(movie_pivot_df.values)

knn_model = NearestNeighbors(metric='cosine', algorithm='brute')


In [19]:
knn_model.fit(movie_pivot_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [55]:
movie_index = np.random.choice(movie_pivot_df_matrix.shape[0])
distance, indeces = knn_model.kneighbors(movie_pivot_df.iloc[movie_index,:].values.reshape(1,-1), n_neighbors=6)

In [56]:
print(len(distance.flatten()))
print(indeces)
print(movie_index)

6
[[108 195 203 107  42 233]]
108


### Display the recommended movies

In [65]:
for i in range(0, len(distance.flatten())):
  if i == 0:
    index_value = [j[i] for j in indeces]
    print('selected movie is: ',movie_pivot_df.index[index_value[0]])
  else:
    index_value = [j[i] for j in indeces]
    print('Related movies: ',movie_pivot_df.index[index_value[0]])

selected movie is:  Dark Knight, The (2008)
Related movies:  Inception (2010)
Related movies:  Iron Man (2008)
Related movies:  Dark Knight Rises, The (2012)
Related movies:  Batman Begins (2005)
Related movies:  Lord of the Rings: The Return of the King, The (2003)
